In [6]:
import pandas as pd
import numpy as np
import pymorphy2
from matplotlib.colors import ListedColormap
from numpy.random import choice
from matplotlib import pyplot as plt

from numpy import genfromtxt

import re

from gensim.models import FastText,KeyedVectors,fasttext

from sklearn import metrics

from scipy import spatial

import torch

from transformers import AutoTokenizer, TFAutoModel, AutoModel

#import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn import cluster

import seaborn as sns
import umap

import time

In [ ]:
path='models/sbert_large_mt_nlu_ru/4'
model = TFAutoModel.from_pretrained(path)
#model = AutoModel.from_pretrained(path)
#tokenizer = AutoTokenizer.from_pretrained(path)


In [17]:
from numpy import genfromtxt
my_data = genfromtxt('faq.csv', delimiter=',',encoding='utf-8',dtype=str,usecols=(0,1))[1:]

## Deprecated

In [ ]:
ft_model= fasttext.FastTextKeyedVectors.load('path')

In [ ]:
def vectorize_sentence_ft(self,sentence):
    txt=sentence.split()
    val=0
    for word in txt:
        val+=self.ft_model[word]

    return val/len(txt)

## FAQ

### Question answering

In [7]:
class SearchEngine():
    def __init__(self,faq,sbert_path=False):
        self.faq=faq
        if sbert_path:
            start = time.time()
            
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            self.sbert_model = AutoModel.from_pretrained(sbert_path)
            self.sbert_model.to(self.device)
            self.sbert_tokenizer = AutoTokenizer.from_pretrained(sbert_path)
            self.faq_embs=np.array([self.sent_vectorizer(sent) for sent in self.faq[:,0]])
            
            end = time.time()
            self.MorphAnalyzer= pymorphy2.MorphAnalyzer()
            print('Sbert model and faq successfully loaded\nPassed time:',round(end - start,2),'s')


    def sent_vectorizer(self,sentence):
    
        encoded_input = self.sbert_tokenizer(sentence,
                                             padding=True,
                                             truncation=True,
                                             max_length=24,
                                             return_tensors='pt').to(self.device)

   #     with torch.no_grad():
        model_output = self.sbert_model(**encoded_input)
            
        #Perform pooling. In this case, mean pooling
        sentence_embedding = self.mean_pooling(model_output, encoded_input['attention_mask'])
        sentence_embedding = np.squeeze(sentence_embedding)
        
        return sentence_embedding.cpu().data.numpy()
    

    

    def mean_pooling(cls,model_output, attention_mask):
        #Mean Pooling - Take attention mask into account for correct averaging
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask



In [10]:
sbert_path = "models/sbert_large_mt_nlu_ru"
faq = genfromtxt('faq.csv', delimiter=',',encoding='utf-8',dtype=str,usecols=(0,1))[1:]

engine = SearchEngine(faq, sbert_path)

Sbert model and faq successfully loaded
Passed time: 11.49 s


In [72]:
def search_faq(self,question,eps,minimal_score,verbose=False):
    '''
    :param question: str
    :param eps: scalar float [0,1]
    :param minimal_score: scalar float [0,1]
    :return: ndarray shape (n,2); [[index1, score1], [index2, score2]..]
    '''

    question_emb=self.sent_vectorizer(question)
    score=np.zeros((self.faq.shape[0],1))

    # обход всего датасета эмбеддингов вопросов
    for i,faq_emb in enumerate(self.faq_embs):
        score[i,0]=1-spatial.distance.cosine( faq_emb,question_emb)

    # индексы эмбеддингов для сортировки
    indeces=np.arange(0,self.faq.shape[0]).reshape((-1,1))

    # сортировка
    faq_logits=np.concatenate([indeces,score],axis=1)
    faq_logits=faq_logits[faq_logits[:, 1].argsort()[::-1]]

    # выислиение количества индексов для вывода
    max_score=faq_logits[0,1]
    display_num=0

    for scr in faq_logits[:,1]:
        if max_score-scr<eps and scr > minimal_score:
            display_num+=1
        else:
            break

    if verbose:
        print('Question: ',question)
        print('---------------------')
        questions_indeces=faq_logits[:,0].astype('int64')
        questions=self.faq[questions_indeces][:display_num,0]
        for i,faq_question in enumerate(questions):
            print('index ',faq_logits[i,0],' score ', faq_logits[i,1],faq_question)



    return faq_logits[:display_num]


In [156]:
def clean_faq(self,faq_logits,verbose=False,plot=False):
    """
    :param self:
    :param faq_logits: ndarray shape (n,2); [[index1, score1], [index2, score2]..]
    :param verbose: bool
    :param plot: bool
    :return: ndarray shape (n,), [index1,index2,index3..]
    """

    questions_indeces=faq_logits[:,0].astype('int64')

    n_clusters=2
    if faq_logits.shape[0]>n_clusters:

        # кластеризация найденных вопросов
        clustering=cluster.KMeans(n_clusters)
        raw_questions=np.concatenate([self.faq_embs[questions_indeces]])
        db_clusters = clustering.fit_predict(raw_questions)

        # вычисление среднего значения точности кластеров
        clusters_score=np.zeros((n_clusters))
        clusters_size=np.zeros((n_clusters))

        for i,logit in enumerate(faq_logits):
            index=db_clusters[i]
            clusters_score[index]+=logit[1]
            clusters_size[index]+=1

        clusters_mean_score=clusters_score/clusters_size

        # вывод кластеризированных вопросов
        max_score_val=clusters_mean_score.max(axis=0)
        max_score_index=np.where(clusters_mean_score==max_score_val)[0]

        true_faq_lofits=faq_logits[np.where(db_clusters==max_score_index)]
        true_questions_indeces=true_faq_lofits[:,0].astype('int64').reshape((1,-1))


        if verbose:
            print('\nCleaned questions')
            print('---------------------')
            true_questions=self.faq[true_questions_indeces][0,:,0]
            for i,faq_question in enumerate(true_questions):
                print('index ',faq_logits[i,0],' score ', faq_logits[i,1],faq_question)


        if plot:
            # вывод кластеризованных вопросов на плоскость
            umap_news=umap.UMAP()
            question_emb=self.sent_vectorizer(question).reshape(1,-1)
            data=np.concatenate([self.faq_embs[questions_indeces][0],question_emb],axis=0)
            umaped_vct=umap_news.fit_transform(data)
            questions=self.faq[questions_indeces][:,0]

            myclr=ListedColormap(choice(list(sns.xkcd_rgb.values()), max(db_clusters)+1)) # Генерируем контрастную карту цветов.
            print('количество кластеров ',max(db_clusters)+1)
            N=15

            fig, ax = plt.subplots(figsize=(N,N))

            ax.scatter(umaped_vct[:-1, 0], umaped_vct[:-1, 1], s=300, c=db_clusters, cmap=myclr)
            ax.scatter(umaped_vct[-1, 0], umaped_vct[-1, 1], s=600, c=1, cmap='rocket')
            for i,xy in enumerate(umaped_vct[:-1]):
                ax.text(xy[0], xy[1],'   '+str(faq_logits[i][1])+' ' +questions[i], fontsize=20,  color='black')

            ax.text(umaped_vct[-1,0], umaped_vct[-1,1],'   Question: '+ question, fontsize=20,  color='black')
            plt.show()

        return true_questions_indeces.astype('int32')[0]
    else:
        return questions_indeces.reshape((1,-1)).astype('int32')


In [134]:
clean_faq(engine,search_faq(engine,'как оформить льготу', 0.2,0.7,True),True)

Question:  как оформить льготу
---------------------
index  9.0  score  0.8306976556777954 Как оформить льготу в Москве?
index  13.0  score  0.811086893081665 Как оформить льготу в Московской области?
index  11.0  score  0.7411444187164307 Какие документы необходимы для оформления/продления/переоформления льготы?

Cleaned questions
---------------------
index  9.0  score  0.8306976556777954 Как оформить льготу в Москве?
index  13.0  score  0.811086893081665 Как оформить льготу в Московской области?


array([[ 9, 13]], dtype=int64)

In [159]:
def questions_diffs(self,questions_indeces,min_score,max_score,verbose=False):

    questions=self.faq[questions_indeces][:,0]

    words_pool_embs=[]
    questions_words_embs=[]
    questions_words=[]
    questions_words_embs_pool=[]


    for question in questions:
        words=question.split()
        questions_words.append(words)
        words_embs=[self.sent_vectorizer(word) for word in words]
        questions_words_embs.append(words_embs)

        words_pool_embs=words_pool_embs+words_embs

    common_words_indx=[]

    # поиск повторяющихся слов
    for i,word1 in enumerate(words_pool_embs):
        word_score=0

        for j,word2 in enumerate(words_pool_embs):
            if i!=j:
                score=1-spatial.distance.cosine(word1, word2)
                if score > word_score:
                    word_score=score

        if word_score>min_score:
            common_words_indx.append(i)

    words_pool_embs=np.array(words_pool_embs)
    common_words_embs=words_pool_embs[common_words_indx]
    questions_diffs_eye=[]

    # повторный проход по вопросам
    for i,question in enumerate(questions_words_embs):
        questions_diffs_eye.append([])

        for word1 in question:
            word_score=0
            for word2 in common_words_embs:
                score=1-spatial.distance.cosine(word1, word2)
                if score > word_score:
                    word_score=score

            if word_score>max_score:
                # если слово повтторяющееся
                questions_diffs_eye[i].append(0)
            else:
                # если слово уникальное
                questions_diffs_eye[i].append(1)

    questions_diffs_eye=np.array(questions_diffs_eye)
    if verbose:

        print('\ndiff indeces ',questions_diffs_eye)
        print('-----------')
    diff_words=[]

    for i,question_eye in enumerate(questions_diffs_eye):
        question_eye=np.array(question_eye)
        indeces=np.where(question_eye==1)[0]
     #   print('indeces ',indeces)

        start=indeces[0]
        end=indeces[-1]+1

        diff_words.append(questions_words[i][start:end])

    diffs= [[" ".join(words)] for words in diff_words]
    for i,diff in enumerate(diffs):
        diff.append(questions_indeces[i])

    return diffs

In [162]:
def compare_answer_diffs(self,diffs_logits,answer):

    answer_emb=self.sent_vectorizer(answer)
    diffs_embs=[self.sent_vectorizer(logit[0]) for logit in diffs_logits]

    index=0
    score=0

    for i,diff_emb in enumerate(diffs_embs):
        diff_score=1-spatial.distance.cosine(diff_emb,answer_emb)
        if score<diff_score:
            index=i
            score=diff_score

    diff_index=diffs_logits[index][1]
    question,answer=self.faq[diff_index]

    return question,diff_index,answer,score



In [142]:
def create_system_question(self,diffs):


        morph = self.MorphAnalyzer
        system_question='Вас интересует '
        n=len(diffs)

        connectors=[]

        if n==2:
            connectors=['или ','?']
        elif n==3:
            connectors = [', ', ' или ','?']

        for i,diff in enumerate(diffs):
            diff_words=diff[0].split()
            q=''
            for word in diff_words:
                p = morph.parse(word)[0]
                if p.tag.POS=='NOUN':
                    q+=p.normal_form+' '
                else:
                    q += word+' '

            system_question+=q+connectors[i]
            
        return system_question

In [ ]:

#noise = ' пук кряк пиво интерал'
#noise2 = ' а то я не умею'
#question = faq['Question'][9]
#question = 'Как оформить льготу '
# question='льгота  от расхода'
# question='Как будут осуществляться начисления если счетчик сломается'
question='Как оформить льготу'

eps = 0.15
minimal_score = 0.7

# ищем похожие вопросы в датасете
# если больше 1 совпадения, кластеризуем и выбираем кластер с наибольшей точностью
faq_logits = search_faq(engine, question, eps, minimal_score, verbose=True)

cleaned_output=clean_faq(engine,faq_logits)

# если после кластеризации больше 1 совпадения
if len(cleaned_output)>1:

    # ищем отличающиеся в вопросах слова
    diffs = questions_diffs(engine,cleaned_output, min_score=0.8, max_score=0.8,verbose=True)
   
    # формируем уточняющий вопрос 
    system_question=create_system_question(engine,diffs)
    print(system_question)

    flag=True
    # сравниеаем ответ пользователя и отличающиеся части вопросов
    while flag:
        answer=input()
        question,question_index,asnwer,score = compare_answer_diffs(engine, diffs, answer)
        if score>0.6:
            flag=False
            print(question,question_index,score)


Question:  Как оформить льготу
---------------------
index  9.0  score  0.8306976556777954 Как оформить льготу в Москве?
index  13.0  score  0.811086893081665 Как оформить льготу в Московской области?
index  11.0  score  0.7411444187164307 Какие документы необходимы для оформления/продления/переоформления льготы?


<ipython-input-159-1490af9c276a>:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  questions_diffs_eye=np.array(questions_diffs_eye)



diff indeces  [list([0, 0, 0, 0, 1]) list([0, 0, 0, 0, 1, 1])]
-----------
Вас интересует Москве? или Московской области? ?


In [11]:
sbert_name = "models/sbert_large_mt_nlu_ru"
# https://rusvectores.org/ru/models/
#fasttext_path = 'weights/model.model'

faq = pd.read_csv('faq.csv')
engine = SearchEngine(faq.values, sbert_name)

Sbert model and faq successfully loaded
Passed time: 10.08 s


In [15]:
diffs

[['Москве?', 9], ['Московской области?', 13]]

In [15]:
engine.faq[output[0]]

array([['Как оформить льготу в Москве?',
        'Для того чтобы использовать льготу при оплате за электроэнергию, ее необходимо оформить/продлить/переоформить вАО «Мосэнергосбыт» одним из удобных способов:В\xa0Едином личном кабинете клиента\xa0вовкладке «Льготы» (с прикреплением необходимых документов);Воспользоваться терминалом видеосвязи (Автоматизированная система «Видеоконсультант») с операторомКонтактного центра, установленным в МФЦ. Часы работы: Пн-Пт с 08:30 до 20:00.В\xa0любом\xa0отделении\xa0АО\xa0«Мосэнергосбыт».В качестве заявителей могут выступать физические лица, проживающие в городе Москве и обладающие правом на льготы по оплате за электроэнергию в соответствии с нормативными правовыми актами Российской Федерации или города Москвы, и обратившиеся с заявлением. Интересы заявителей, могут представлять законные представители заявителей или иные лица, уполномоченные заявителем в установленном порядке.В соответствии с законодательством РФ льготы по оплате электроэнергии предо

In [ ]:
%%time
noise=' пук кряк пиво интерал'
noise2=' а то я не умею'
question=faq['Question'][9] 
question='Как оформить льготу '
#question='льгота  от расхода'
#question='Как будут осуществляться начисления если счетчик сломается'


eps=0.15
minimal_score=0.7
beta=1.5

modes=['fasttext','sbert']
mode=modes[1]

output=find_faq(engine,question,eps,minimal_score,mode=mode,verbose=True,plot=False)
diffs=questions_diff(engine.sent_vectorizer,questions_indeces=output,min_score=0.8,max_score=0.8,verbose=0)
print('\nQuestions difference')
print('--------------')
print(diffs)

In [ ]:
%%time
answer='московской области'
question=diff_search(engine,diffs,answer,mode)
print(question)

In [ ]:
vect1=engine.sent_vectorizer('москва')
vect2=engine.sent_vectorizer('московская область')
vect3=engine.sent_vectorizer('город')
engine.cos_dist(vect1,vect3)

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
morph.parse('области')[0].normal_form
# ner

In [ ]:
questions=['Сборка простой люстры',
'Сборка сложной люстры',
'Установка простой люстры',
'Установка сложной люстры',
'Установка светильника типа Армстронг',
'Установка светильника настенного, бра',
'Установка точечного светильника',
'Подключение светильника Выход',
'Подключени трансформатора для галогенных ламп',
'Установка антивандального светильника']


In [ ]:
q1=questions[:2]
q1

In [ ]:
q2=questions[4:7]
q2_words=[]
morph = pymorphy2.MorphAnalyzer()

for q in q2:
    words=q.split()
    for word in words:
        p = morph.parse(word)[0] 
        q2_words.append(p.normal_form)
        
q2_words=set(q2_words)
q2

In [ ]:
q2_words

In [ ]:


word = "светильником"
p = morph.parse(word)[0]  # Делаем полный разбор, и берем первый вариант разбора (условно "самый вероятный", но не факт что правильный)
print(p.normal_form)  # стать